In [1]:
import os
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd, YearEnd
import wrds

In [2]:
fundamentals  = pd.read_csv('full_fundamental_dataset.csv')

/var/folders/zv/y4lfwq1s277d8l9hk50_nyc40000gn/T/ipykernel_57776/2864504037.py:1: DtypeWarning: Columns (62) have mixed types. Specify dtype option on import or set low_memory=False.
  fundamentals  = pd.read_csv('full_fundamental_dataset.csv')


In [3]:
fundamentals = fundamentals.sort_values(['permno', 'jdate']).rename(columns={'jdate': 'fund_date'})
fundamentals['fund_date'] = pd.to_datetime(fundamentals['fund_date'])

In [4]:
conn=wrds.Connection()
crsp_m = conn.raw_sql("""
    SELECT
        a.permno,
        a.permco,
        a.date,
        b.shrcd,
        b.exchcd,
        a.ret,
        a.retx,
        a.shrout,
        a.prc
    FROM crsp.msf AS a
    LEFT JOIN crsp.msenames AS b
      ON a.permno = b.permno
      AND b.namedt <= a.date
      AND a.date <= b.nameendt
    WHERE a.date BETWEEN '01/01/1959' AND '12/31/2024'
      AND b.exchcd BETWEEN 1 AND 3
      AND b.shrcd in (10, 11)
    """, date_cols=['date'])

Loading library list...
Done


In [5]:
crsp_m[['permco', 'permno', 'shrcd', 'exchcd', 'shrout']] = crsp_m[['permco', 'permno', 'shrcd', 'exchcd', 'shrout']].astype(int)
crsp_m = crsp_m.sort_values(['permno', 'date'])

In [6]:
dlret = conn.raw_sql("""
    SELECT permno, dlret, dlstdt 
    FROM crsp.msedelist
    """, date_cols=['dlstdt'])
dlret['permno'] = dlret['permno'].astype(int)
dlret['jdate'] = dlret['dlstdt'] + MonthEnd(0)

In [7]:
crsp_m = pd.merge_asof(
    crsp_m.sort_values('date'),
    dlret.sort_values('jdate'),
    left_on='date',
    right_on='jdate',
    by='permno',
    direction='backward'
)
crsp_m['dlret'] = crsp_m['dlret'].fillna(0)
crsp_m['ret'] = crsp_m['ret'].fillna(0)

crsp_m['retadj'] = (1 + crsp_m['ret']) * (1 + crsp_m['dlret']) - 1
crsp_m['me'] = crsp_m['prc'].abs() * crsp_m['shrout']

In [8]:
monthly_dataset = pd.merge_asof(
    crsp_m.sort_values('date'),
    fundamentals.sort_values('fund_date'),
    left_on='date',
    right_on='fund_date',
    by='permno',
    direction='backward'
)

In [9]:
monthly_dataset.to_csv('monthly_dataset.csv', index=False)

In [10]:
monthly_dataset.to_parquet('monthly_dataset.parquet', index=False)